In [10]:
%load_ext autoreload

In [11]:
%autoreload

In [1]:
import sys
print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(["/home/mathia/PycharmProjects/neslearn-git/","/home/peer/PycharmProjects/neslearng/"])

Python 3.4.2 (default, Oct  8 2014, 10:45:20) 
[GCC 4.9.1] on linux


In [2]:
from hsa.tree_search import fceux_a_star
from tqdm import tqdm_notebook
from functools import partial
from hsa.tree_search import heuristics

In [3]:
from extern.fceux_learningenv.nes_python_interface.nes_python_interface import NESInterface
from hsa.machine_constants import mario_rom_location

nes = NESInterface(mario_rom_location,eb_compatible=False)

In [17]:
config = fceux_a_star.ConfigPack(nes, render_search=False, nr_nodes_to_expand=20)

In [23]:
w = [ 36.0454073 ,   4.36050218,  17.36822846,  -0.58672081,  10.96647671]
action_repeat = int(w[0])
#ram, x_pos_resolution=1, x_pos_factor=1, shortcut_factor=1, height_factor=0, punishment_death=True,speed_factor=0
heuristic_params = {"x_pos_resolution": w[1],
                    "x_pos_factor": w[2],
                    "shortcut_factor": w[2],
                    "height_factor": w[3],
                    "speed_factor": w[4]}
heuristic = partial(heuristics.combined, **heuristic_params)
config = fceux_a_star.ConfigPack(nes, heuristic=heuristic, action_repeat=action_repeat, nr_nodes_to_expand=200)

In [24]:
start = fceux_a_star.make_start(nes,config.heuristic, reset=True)

In [25]:
graph = fceux_a_star.AStarGraph(config)

In [26]:
best = None
for current_best in tqdm_notebook(fceux_a_star.best_first(start,graph,config),total=config.nr_nodes_to_expand):
    best = current_best
    nes.render()
    #fceux_a_star.render_node(current_best,nes)
best

MarioNode(potential:6412.85640454, action:0)

In [13]:
nes.restoreState(start.state)
for action, repeat in list(reversed(list(fceux_a_star.walk_nodes(best)))):
    for i in range(repeat):
        nes.act(action)
        nes.render()


## Debug analysis

In [9]:
graph.get_vertex_neighbours(start)

[MarioNode(id=2149157194268967603, potential=40, recv_reward=0, state=array([70, 67, 83, ...,  0,  0,  0], dtype=uint8)),
 MarioNode(id=2149157194270050128, potential=40, recv_reward=0, state=array([70, 67, 83, ...,  0,  0,  0], dtype=uint8)),
 MarioNode(id=2149157194271132653, potential=40, recv_reward=0, state=array([70, 67, 83, ...,  0,  0,  0], dtype=uint8)),
 MarioNode(id=2149157194251647203, potential=40, recv_reward=0, state=array([70, 67, 83, ...,  0,  0,  0], dtype=uint8)),
 MarioNode(id=2149157194407530803, potential=70, recv_reward=150, state=array([70, 67, 83, ...,  0,  0,  0], dtype=uint8)),
 MarioNode(id=2149157194338249203, potential=6, recv_reward=-165, state=array([70, 67, 83, ...,  0,  0,  0], dtype=uint8)),
 MarioNode(id=2149157194303608403, potential=40, recv_reward=0, state=array([70, 67, 83, ...,  0,  0,  0], dtype=uint8)),
 MarioNode(id=2149157194252729728, potential=40, recv_reward=0, state=array([70, 67, 83, ...,  0,  0,  0], dtype=uint8)),
 MarioNode(id=214915

In [14]:
%prun graph.get_vertex_neighbours(start)

In [8]:
it_for_time = fceux_a_star.best_first(start,graph,config)
%timeit next(it_for_time)

1 loop, best of 3: 859 ms per loop


In [10]:
org_render = config.render_search
config.render_search = False
it_for_time = fceux_a_star.best_first(start,graph,config)
%timeit next(it_for_time)
config.render_search = org_render

1 loop, best of 3: 843 ms per loop


In [16]:
len(config.actions) * config.action_repeat

600

In [13]:
fceux_a_star.make_start(nes,config.heuristic)>fceux_a_star.make_start(nes,config.heuristic)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()